# LinkedIn - LinkedIn Messaging: User Engagement Insights

```SQL
CREATE TABLE fct_messages (
    message_id integer,
    user_id integer,
    message_sent_date date
);

INSERT INTO fct_messages (message_id, user_id, message_sent_date)
VALUES
    (1, 1, '2024-04-01'), (2, 1, '2024-04-02'), (3, 1, '2024-04-03'),
    (4, 1, '2024-04-04'), (5, 1, '2024-04-05'), (6, 1, '2024-04-06'),
    (7, 1, '2024-04-07'), (8, 1, '2024-04-08'), (9, 1, '2024-04-09'),
    (10, 1, '2024-04-10'), (11, 1, '2024-04-11'), (12, 1, '2024-04-12'),
    (13, 1, '2024-04-13'), (14, 1, '2024-04-14'), (15, 1, '2024-04-15'),
    (16, 1, '2024-04-16'), (17, 1, '2024-04-17'), (18, 1, '2024-04-18'),
    (19, 1, '2024-04-19'), (20, 1, '2024-04-20'), (21, 1, '2024-04-21'),
    (22, 1, '2024-04-22'), (23, 1, '2024-04-23'), (24, 1, '2024-04-24'),
    (25, 1, '2024-04-25'), (26, 1, '2024-04-26'), (27, 1, '2024-04-27'),
    (28, 1, '2024-04-28'), (29, 1, '2024-04-29'), (30, 1, '2024-04-30'),
    (31, 1, '2024-04-01'), (32, 1, '2024-04-02'), (33, 1, '2024-04-03'),
    (34, 1, '2024-04-04'), (35, 1, '2024-04-05'), (36, 1, '2024-04-06'),
    (37, 1, '2024-04-07'), (38, 1, '2024-04-08'), (39, 1, '2024-04-09'),
    (40, 1, '2024-04-10'), (41, 1, '2024-04-11'), (42, 1, '2024-04-12'),
    (43, 1, '2024-04-13'), (44, 1, '2024-04-14'), (45, 1, '2024-04-15'),
    (46, 1, '2024-04-16'), (47, 1, '2024-04-17'), (48, 1, '2024-04-18'),
    (49, 1, '2024-04-19'), (50, 1, '2024-04-20'), (51, 1, '2024-04-21'),
    (52, 1, '2024-04-22'), (53, 1, '2024-04-23'), (54, 1, '2024-04-24'),
    (55, 1, '2024-04-25'), (56, 2, '2024-04-10'), (57, 2, '2024-04-18'),
    (58, 2, '2024-04-25'), (59, 3, '2024-04-12'), (60, 3, '2024-04-22'),
    (61, 4, '2024-05-05'), (62, 5, '2024-06-10'), (63, 6, '2024-03-30'),
    (64, 7, '2024-07-01'), (65, 2, '2024-05-15');
```

In [4]:
import pandas as pd 
import numpy as np

In [5]:
df_messages = pd.read_csv('Data/012/fct_messages.csv')
df_messages.head()

,message_id,user_id,message_sent_date
0,1,1,2024-04-01
1,2,1,2024-04-02
2,3,1,2024-04-03
3,4,1,2024-04-04
4,5,1,2024-04-05


# Preguntas 1

### ¿Cuál es el número total de mensajes enviados durante abril de 2024? Esta información nos ayudará a cuantificar el compromiso (engagement) general como una línea base para mejoras focalizadas en el producto.

In [7]:
df_messages['message_sent_date'] = pd.to_datetime(df_messages['message_sent_date'])
df_abril = df_messages[
    (df_messages['message_sent_date'].dt.year == 2024) &
    (df_messages['message_sent_date'].dt.month == 4)
]

resultado = df_abril['message_id'].count()

resultado

np.int64(60)

```SQL
SELECT
    COUNT(fct_messages.message_id)
FROM fct_messages
WHERE EXTRACT (YEAR FROM message_sent_date) = 2024
AND EXTRACT (MONTH FROM message_sent_date) = 4;
```

# Pregunta 2

### ¿Cuál es el promedio de mensajes enviados por usuario durante abril de 2024? Redondea tu resultado al número entero más cercano. Esta métrica proporciona información sobre los niveles de compromiso individual para perfeccionar nuestras funciones de comunicación.

In [10]:
mensajes_por_usuario = df_abril.groupby('user_id')['message_id'].count()

resultado = mensajes_por_usuario.mean().round(0)

resultado

np.float64(20.0)

```SQL
SELECT
    ROUND(AVG(conteo_por_usuario), 0) AS promedio_mensajes
FROM(
    SELECT
        user_id,
        COUNT(message_id) AS conteo_por_usuario
    FROM fct_messages
    WHERE EXTRACT (YEAR FROM message_sent_date) = 2024
    AND EXTRACT (MONTH FROM message_sent_date) = 4
    GROUP BY user_id
) AS subquery;
```

# Pregunta 3

### ¿Qué porcentaje de usuarios envió más de 50 mensajes durante abril de 2024? Este cálculo ayudará a identificar a los usuarios altamente comprometidos y servirá de base para recomendaciones orientadas a mejorar las interacciones de mensajería.

In [12]:
conteo_abril = df_abril.groupby('user_id')['message_id'].count()

total_usuarios = len(conteo_abril)

usuarios_top = len(conteo_abril[conteo_abril > 50])

porcentaje = (usuarios_top / total_usuarios) * 100

porcentaje

33.33333333333333

```SQL
SELECT
    (COUNT(user_id) * 100.0 / (SELECT COUNT(DISTINCT user_id)
                               FROM fct_messages
                               WHERE EXTRACT(MONTH FROM message_sent_date) = 4
                                 AND EXTRACT(YEAR FROM message_sent_date) = 2024)) AS porcentaje_usuarios_top
FROM (
    SELECT user_id
    FROM fct_messages
    WHERE EXTRACT(MONTH FROM message_sent_date) = 4
      AND EXTRACT(YEAR FROM message_sent_date) = 2024
    GROUP BY user_id
    HAVING COUNT(message_id) > 50
) AS usuarios_intensivos;
```